In [1]:
import numpy as np
from IPython import display
import gpiozero
import time

In [2]:
# Defining GPIO inputs and outputs
pwm_l = gpiozero.PWMOutputDevice(pin=18,active_high=True,initial_value=0,frequency=50000)
dir_l = gpiozero.OutputDevice(pin=23)

pwm_r = gpiozero.PWMOutputDevice(pin=15,active_high=True,initial_value=0,frequency=50000)
dir_r = gpiozero.OutputDevice(pin=14)

encoder_l = gpiozero.RotaryEncoder(a=5, b=6,max_steps=100000) 
encoder_r = gpiozero.RotaryEncoder(a=25, b=8,max_steps=100000) 


# add sensors

In [3]:
# Defining robot model class
class DiffDriveRobot:
    
    def __init__(self, dt=0.1, wheel_radius=0.028, wheel_sep=0.101):
        
        self.x = 0.0 # y-position
        self.y = 0.0 # y-position 
        self.th = 0.0 # orientation
        
        self.wl = 0.0 # rotational velocity left wheel
        self.wr = 0.0 # rotational velocity right wheel
        
        self.steps_l = 0 # total right encoder steps
        self.steps_r = 0 # total left encoder steps
        
        self.dt = dt
        self.r = wheel_radius
        self.l = wheel_sep

    
    # Retrieving step count from encoders. 
    # If encoder steps higher, then wheel has gone clockwise.
    # This is forward movement for right wheel and backwards for left wheel, 
    # so multiply by -1 for left.
    def get_steps(self, encoder, side):
        if (side == 0):     # zero for left side
            steps_curr = encoder.steps - self.steps_l
            self.steps_l = encoder.steps
            return (steps_curr * -1)
        elif (side == 1):
            steps_curr = encoder.steps - self.steps_r
            self.steps_r = encoder.steps
            return steps_curr
        
    # Calculate motor angular velocity from shaft encoder feedback
    def wheel_speed(self, steps):

        N = 64 # Encoder state transitions
        RATIO = 114.7 # Gear ratio
        
        steps_ps = steps/self.dt
        rps_encoder = steps_ps/N
        rps_wheel = rps_encoder/RATIO
        w = 2*np.pi*rps_wheel
        
        return w
    
    # Velocity motion model
    # Calculate base linear and angular velocity from left and right wheel angular velocities
    def base_velocity(self, wl, wr):
        
        v = (wl*self.r + wr*self.r)/2.0
        w = (wl - wr)/self.l
        
        return v, w
    
    # Kinematic motion model
    def pose_update(self, duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r):
        
        # Get current steps
        steps_curr_l = self.get_steps(encoder_l, 0)
        steps_curr_r = self.get_steps(encoder_r, 1)
        
        # Update angular velocities
        self.wl = self.wheel_speed(steps_curr_l)
        self.wr = self.wheel_speed(steps_curr_r)
        
        # Update motor output
        pwm_l.value = duty_cycle_l
        pwm_r.value = duty_cycle_r
        
        # Update robot position
        v, w = self.base_velocity(self.wl,self.wr)
        self.x = self.x + self.dt*v*np.cos(self.th)
        self.y = self.y + self.dt*v*np.sin(self.th)
        self.th = self.th + w*self.dt
        
        
        return self.x, self.y, self.th
        

In [4]:
# Defining robot velocity controller class
class RobotController:
    
    def __init__(self, Kp=1, Ki=0.25, wheel_radius=0.028, wheel_sep=0.101):
        
        self.Kp = Kp
        self.Ki = Ki
        self.r = wheel_radius
        self.l = wheel_sep
        self.e_sum_l = 0 # error term of left wheel
        self.e_sum_r = 0 # error term of right wheel
        
    # Implement duty cycle PI control given target and actual angular velocity
    def p_control(self,w_desired,w_measured,e_sum):
        
        duty_cycle = min(max(-1,self.Kp*(w_desired-w_measured) + self.Ki*e_sum),1)
        e_sum = e_sum + (w_desired-w_measured)
        
        return duty_cycle, e_sum
        
        
    # Calculate duty cycles from target linear and angular velocity 
    def drive(self,v_desired,w_desired,wl,wr):
        
        wl_desired = v_desired/self.r + self.l*w_desired/2 
        wr_desired = v_desired/self.r - self.l*w_desired/2
        
        duty_cycle_l,self.e_sum_l = self.p_control(wl_desired,wl,self.e_sum_l)
        duty_cycle_r,self.e_sum_r = self.p_control(wr_desired,wr,self.e_sum_r)
        
        
        return duty_cycle_l, duty_cycle_r
        

In [5]:
# Defining robot path planner class (without obstacle avoidance)
class TentaclePlanner:
    
    def __init__(self,dt=0.1,steps=5,alpha=1,beta=0.1):
        
        self.dt = dt
        self.steps = steps
        # Tentacles are possible trajectories to follow
        self.tentacles = [(0.0,0.5),(0.0,-0.5),(0.1,1.0),(0.1,-1.0),(0.1,0.5),(0.1,-0.5),(0.1,0.0),(0.0,0.0)]
        
        self.alpha = alpha
        self.beta = beta
    
    # Play a trajectory and evaluate where you'd end up
    def roll_out(self,v,w,goal_x,goal_y,goal_th,x,y,th):
        
        for j in range(self.steps):
        
            x = x + self.dt*v*np.cos(th)
            y = y + self.dt*v*np.sin(th)
            th = (th + w*self.dt)
        
        # Wrap angle error -pi,pi
        e_th = goal_th-th
        e_th = np.arctan2(np.sin(e_th),np.cos(e_th))
        
        cost = self.alpha*((goal_x-x)**2 + (goal_y-y)**2) + self.beta*(e_th**2)
        
        return cost
    # Choose trajectory that will get you closest to the goal
    def plan(self,goal_x,goal_y,goal_th,x,y,th):
        
        costs = []
        for v,w in self.tentacles:
            costs.append(self.roll_out(v,w,goal_x,goal_y,goal_th,x,y,th))
        
        best_idx = np.argmin(costs)
        
        return self.tentacles[best_idx]
        

In [6]:
# Initialising models
dt = 0.1
r = 0.028
sep = 0.101


robot = DiffDriveRobot(dt = dt, wheel_radius = r, wheel_sep = sep)
controller = RobotController(Kp = 1, Ki = 0.25, wheel_radius = r, wheel_sep = sep)
planner = TentaclePlanner(dt = dt, steps = 5, alpha = 1, beta = 1e-5)

In [7]:
# # Example controller

# for i in range(300):
    
# #     if i < 100: # drive in circular path (turn left) for 10 s
# #         duty_cycle_l,duty_cycle_r = controller.drive(0.1,1,robot.wl,robot.wr)
#     if i < 200: # drive in circular path (turn right) for 10 s
#         duty_cycle_l,duty_cycle_r = controller.drive(0.1,-1,robot.wl,robot.wr)
#     else: # stop
#         duty_cycle_l,duty_cycle_r = (0,0)
        
#     # Drive robot motion - send duty cycle command to robot
#     x,y,th  = robot.pose_update(duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r)
#     time.sleep(dt)
    
            
   
#     print('Duty cycle left:',duty_cycle_l,'Duty cycle right:',duty_cycle_r)
#     print('Direction left:',dir_l.value,'Direction right:',dir_r.value)
#     print('Velocity left:',robot.wl,'Velocity right:',robot.wr)
#     print('Steps left:',robot.steps_l,'Steps right:',robot.steps_r,'\n')
    

In [8]:
# Navigation using path planning
poses = []
velocities = []
duty_cycle_commands = []

goal_x = 0.3
goal_y = 0.3
goal_th = 0

for i in range(200):

    # Plan using tentacles
    v, w = planner.plan(goal_x, goal_y, goal_th, robot.x, robot.y, robot.th)
    
    # Calculate duty cycle from required velocities
    duty_cycle_l,duty_cycle_r = controller.drive(v, w, robot.wl, robot.wr)
    
    print('Duty cycle left:',duty_cycle_l,'Duty cycle right:',duty_cycle_r)
    print('Velocity left:',robot.wl,'Velocity right:',robot.wr)
    print('Velocity goal:',v, 'Angular velocity goal:',w)
    print('Steps left:',robot.steps_l,'Steps right:',robot.steps_r)
    
    # Output duty cycles to robot
    x, y, th = robot.pose_update(duty_cycle_l, duty_cycle_r, encoder_l, encoder_r, pwm_l, pwm_r)
    time.sleep(dt)
    
    print('x:',x,'y:',y,'th:',th,'\n')
    
    # Log data
    poses.append([x,y,th])
    duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
    velocities.append([robot.wl,robot.wr])

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.0
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 0 Steps right: 0
x: 0.0 y: 0.0 th: 0.0 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.0
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 0 Steps right: 0
x: 0.0005631996420177865 y: 0.0 th: -0.6694870703211824 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.13694824122013047 Velocity right: 0.5392336998042637
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 16 Steps right: 63
x: 0.0006101812819227927 y: -3.71821923166628e-05 th: -0.711859669708599 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.04279632538129077
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 16 Steps right: 68
x: 0.0006373999430401205 y: -6.0665566347865906e-05 th: -0.7711813088509822 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.01711853015251631 Velocity right: 0.04279632

x: 0.0006488153248556517 y: -0.0007350463659519281 th: -2.135579009125797 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.025677795228774464
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 68 Steps right: 184
x: 0.0006488153248556517 y: -0.0007350463659519281 th: -2.135579009125797 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.0
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 68 Steps right: 184
x: 0.0006808836883512568 y: -0.0006844359917550234 th: -2.211849688023147 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.059914855533807085 Velocity right: 0.01711853015251631
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 75 Steps right: 186
x: 0.0006665510961149696 y: -0.0007036438856002693 th: -2.2287987277781136 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.01711853015251631
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 75 Steps right: 188


x: -0.0001705070400943609 y: -0.0006020663646221782 th: -4.245734458619143 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.03423706030503262 Velocity right: 0.08559265076258155
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 154 Steps right: 347
x: -0.00019746279556536746 y: -0.0005485577002712128 th: -4.355903217026427 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.03423706030503262 Velocity right: 0.0770333856863234
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 158 Steps right: 356
x: -0.00022255392212394562 y: -0.0004811801606629825 th: -4.49149553506616 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.04279632538129077 Velocity right: 0.0941519158388397
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 163 Steps right: 367
x: -0.00026718718934833996 y: -0.0002824193927571424 th: -4.652511412738343 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.008559265076258154 Velocity right: 0.1540667713726468
Velocity

x: 0.0017578385057273253 y: 0.0009059120360105422 th: -7.703338568632336 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.05135559045754893 Velocity right: 0.05135559045754893
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 197 Steps right: 712
x: 0.001768628496275966 y: 0.0008348284684326731 th: -7.754185687897237 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.05135559045754893
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 197 Steps right: 718
x: 0.0017710162322073565 y: 0.000810981768382061 th: -7.771134727652203 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.01711853015251631
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 197 Steps right: 720
x: 0.0017749826995296532 y: 0.0007632142827698915 th: -7.805032807162137 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.03423706030503262
Velocity goal: 0.0 Angular velocity goal: -0.5
Steps left: 197 Steps 

x: 0.001122213503429358 y: 0.0003445588155131594 th: -10.06772961445018 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.008559265076258154 Velocity right: 0.03423706030503262
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 290 Steps right: 898
x: 0.0011509844811323125 y: 0.00032300527262746635 th: -10.211796452367397 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.08559265076258155 Velocity right: 0.059914855533807085
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 300 Steps right: 905
x: 0.001125605982388286 y: 0.00034846614791502774 th: -10.27111809150978 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.01711853015251631 Velocity right: 0.04279632538129077
Velocity goal: 0.1 Angular velocity goal: -1.0
Steps left: 302 Steps right: 910
x: 0.001093840165737441 y: 0.0003843604153853712 th: -10.32196521077468 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.008559265076258154 Velocity right: 0.04279632538129077
Velocity goal

x: 0.0011486621153836564 y: 0.00019473167912340063 th: -12.499916819287892 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.05135559045754893 Velocity right: 0.025677795228774464
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 437 Steps right: 1038
x: 0.0012204012461951813 y: 0.0001995060467301279 th: -12.618560097572658 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.03423706030503262 Velocity right: 0.08559265076258155
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 441 Steps right: 1048
x: 0.001256301212841448 y: 0.000197630743099713 th: -12.643983657205109 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.025677795228774464
Velocity goal: 0.1 Angular velocity goal: 1.0
Steps left: 441 Steps right: 1051
x: 0.0010890446435988996 y: 0.00021063816283333007 th: -12.77957597524484 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.12838897614387232 Velocity right: 0.008559265076258154
Velocity goal: 0.1 Angular 

x: 0.0007461168768166559 y: -0.00036650387903534604 th: -15.6948108130991 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.025677795228774464
Velocity goal: 0.0 Angular velocity goal: 0.0
Steps left: 596 Steps right: 1256
x: 0.0007341349421444758 y: -0.0003666614799778824 th: -15.703285332976582 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: 0.0 Velocity right: 0.008559265076258154
Velocity goal: 0.0 Angular velocity goal: 0.0
Steps left: 596 Steps right: 1257
x: 0.00067422074217013 y: -0.0003669417567537271 th: -15.813454091383866 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.03423706030503262 Velocity right: 0.0770333856863234
Velocity goal: 0.0 Angular velocity goal: 0.0
Steps left: 600 Steps right: 1266
x: 0.0007576352466588704 y: -0.00037577400854058105 th: -15.957520929301083 

Duty cycle left: 1 Duty cycle right: 1
Velocity left: -0.10271118091509786 Velocity right: 0.04279632538129077
Velocity goal: 0.0 Angular velocity goal: 0

In [9]:
pwm_l.off()
pwm_r.off()

In [10]:
# Defining robot path planner class (with obstacle avoidance)

In [11]:
# Run robot using predefined coordinates